In [5]:
import os

import numpy as np
import astropy.io.fits as fits

import sacc

In [6]:
# Original sacc file
original_sacc_file = "../../../star-challenge/gaussian-sims-srd-sample/data-vector/twopoint_theory_real_with_cov.sacc"

# Name of new sacc file that will be created
output_sacc_file = "../../../star-challenge/gaussian-sims-srd-sample/data-vector/twopoint_theory_real_with_cov_firecrown_theory.sacc"

# Name of sacc file to take the covariance from
cov_sacc_file = original_sacc_file

# Directory with the firecrown datablock output
datablock_dir = "../cosmosis/output/gaussian_sims_3x2pt_real/"

In [7]:
original_sacc_data = sacc.Sacc.load_fits(original_sacc_file)
cov_sacc_data = sacc.Sacc.load_fits(cov_sacc_file)

new_sacc_data = original_sacc_data.copy()


In [ ]:
all_cov_indices = []

for data_type in new_sacc_data.get_data_types():
    print(data_type)
    tracers = original_sacc_data.get_tracer_combinations(data_type)
    for tracer0, tracer1 in tracers:
        print((tracer0, tracer1))
        theory = np.loadtxt(os.path.join(datablock_dir, f"datablock/data_vector/theory_{data_type}_{tracer0}_{tracer1}.txt"))

        indices = new_sacc_data.indices(data_type=data_type, tracers=(tracer0, tracer1))
        cov_indices = cov_sacc_data.indices(data_type=data_type, tracers=(tracer0, tracer1))
        all_cov_indices += list(indices)
        assert len(indices) == theory.size
        for i, idx in enumerate(indices):
            new_sacc_data.data[idx].value = theory[i]

# We don't need to deal with the covariance now.
# new_covariance = cov_sacc_data.covariance.keeping_indices(all_cov_indices)
# new_sacc_data.add_covariance(new_covariance.covmat)

new_sacc_data.save_fits(output_sacc_file, overwrite=True)